<a href="https://colab.research.google.com/github/JasonD28/CS-175/blob/master/model/faster_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pydicom

     |████████████████████████████████| 35.5MB 87kB/s 


In [3]:
%%shell
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi
cp PythonAPI/pycocotools /usr/local/lib/python3.6/dist-packages/ -r

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 28.25 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [4]:
%%shell
git clone https://github.com/JasonD28/CS-175.git
cd CS-175
cp model/dataset.py  ../

Cloning into 'CS-175'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 164 (delta 83), reused 47 (delta 18), pack-reused 0
Receiving objects: 100% (164/164), 1.43 MiB | 6.03 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [5]:
%%shell

# Using PyTorch helper functions to simplify training
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.5.1

cp references/detection/utils.py ../
cp references/detection/engine.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8389 (delta 0), reused 1 (delta 0), pack-reused 8384
Receiving objects: 100% (8389/8389), 10.25 MiB | 21.64 MiB/s, done.
Resolving deltas: 100% (5775/5775), done.
Branch 'v0.5.1' set up to track remote branch 'v0.5.1' from 'origin'.
Switched to a new branch 'v0.5.1'


In [0]:
import torch
import utils
from torch import optim
import torchvision
from dataset import PneumoniaDataset
from engine import train_one_epoch, evaluate
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

root = "drive/My Drive/cs-175-project/cs-175-data"

In [0]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=3)

In [3]:
# model.load_state_dict(torch.load('drive/My Drive/cs-175-project/model_checkpoints/sixth_try_model.pt'))

<All keys matched successfully>

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
dataset = PneumoniaDataset(root, True)
test_dataset = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-1000])
test_dataset = torch.utils.data.Subset(test_dataset, indices[-1000:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=20, shuffle=True, num_workers=8,
        collate_fn=utils.collate_fn)

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0005,
                            momentum=0.9, weight_decay=0.0005)
lr_step = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=5,
                                               gamma=0.1)


In [4]:
for epoch in range(7):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=20)
    lr_step.step()
    torch.save(model.state_dict(), 'drive/My Drive/cs-175-project/model_checkpoints/seventh_try_model.pt')
    evaluate(model, test_data_loader, device=device)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch: [0]  [   0/1285]  eta: 1:36:14  lr: 0.000001  loss: 2.8497 (2.8497)  loss_classifier: 1.0852 (1.0852)  loss_box_reg: 0.0088 (0.0088)  loss_objectness: 1.2475 (1.2475)  loss_rpn_box_reg: 0.5081 (0.5081)  time: 4.4934  data: 1.9549  max mem: 12644
Epoch: [0]  [  20/1285]  eta: 0:50:49  lr: 0.000011  loss: 2.9561 (2.9219)  loss_classifier: 1.0172 (1.0058)  loss_box_reg: 0.0058 (0.0066)  loss_objectness: 1.3429 (1.3401)  loss_rpn_box_reg: 0.5668 (0.5694)  time: 2.3063  data: 0.0571  max mem: 12903
Epoch: [0]  [  40/1285]  eta: 0:48:51  lr: 0.000021  loss: 1.8599 (2.4558)  loss_classifier: 0.6188 (0.8255)  loss_box_reg: 0.0068 (0.0075)  loss_objectness: 0.7700 (1.0899)  loss_rpn_box_reg: 0.4836 (0.5329)  time: 2.2968  data: 0.0468  max mem: 12903
Epoch: [0]  [  60/1285]  eta: 0:47:41  lr: 0.000031  loss: 0.7978 (1.9372)  loss_classifier: 0.2356 (0.6399)  loss_box_reg: 0.0043 (0.0066)  loss_objectness: 0.1193 (0.7932)  loss_rpn_box_reg: 0.4201 (0.4976)  time: 2.2977  data: 0.0579  max

In [0]:
model2 = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features2 = model2.roi_heads.box_predictor.cls_score.in_features
model2.roi_heads.box_predictor = FastRCNNPredictor(in_features2, num_classes=3)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model2.to(device)
dataset2 = PneumoniaDataset(root, True)
test_dataset2 = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset2)).tolist()
dataset2 = torch.utils.data.Subset(dataset2, indices[:-50])
test_dataset2 = torch.utils.data.Subset(test_dataset2, indices[-50:])

data_loader2 = torch.utils.data.DataLoader(
        dataset2, batch_size=10, shuffle=True, num_workers=6,
        collate_fn=utils.collate_fn)

test_data_loader2 = torch.utils.data.DataLoader(
    test_dataset2, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)
params = [p for p in model2.parameters() if p.requires_grad]
optimizer2 = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
lr_step2 = torch.optim.lr_scheduler.StepLR(optimizer2,
                                               step_size=5,
                                               gamma=0.1)
for epoch in range(2):
    train_one_epoch(model2, optimizer2, data_loader2, device, epoch, print_freq=40)
    lr_step2.step()
    torch.save(model2.state_dict(), 'drive/My Drive/cs-175-project/model_checkpoints/second_try_model.pt')
    evaluate(model2, test_data_loader2, device=device)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch: [0]  [   0/2664]  eta: 5:33:19  lr: 0.000010  loss: 1.9302 (1.9302)  loss_classifier: 0.4352 (0.4352)  loss_box_reg: 0.0099 (0.0099)  loss_objectness: 1.0406 (1.0406)  loss_rpn_box_reg: 0.4444 (0.4444)  time: 7.5074  data: 5.4561  max mem: 7168
Epoch: [0]  [  40/2664]  eta: 1:32:11  lr: 0.000210  loss: 0.3582 (1.0259)  loss_classifier: 0.0314 (0.1667)  loss_box_reg: 0.0024 (0.0052)  loss_objectness: 0.0732 (0.4746)  loss_rpn_box_reg: 0.2473 (0.3794)  time: 1.9481  data: 0.0343  max mem: 7434
Epoch: [0]  [  80/2664]  eta: 1:27:39  lr: 0.000410  loss: 0.1192 (0.5948)  loss_classifier: 0.0189 (0.0933)  loss_box_reg: 0.0057 (0.0057)  loss_objectness: 0.0177 (0.2568)  loss_rpn_box_reg: 0.0740 (0.2390)  time: 1.9622  data: 0.0323  max mem: 7434
Epoch: [0]  [ 120/2664]  eta: 1:25:13  lr: 0.000609  loss: 0.0866 (0.4308)  loss_classifier: 0.0256 (0.0702)  loss_box_reg: 0.0098 (0.0069)  loss_objectness: 0.0157 (0.1786)  loss_rpn_box_reg: 0.0358 (0.1751)  time: 1.9600  data: 0.0336  max me

Process Process-12:
Process Process-11:
Process Process-13:
Process Process-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/util

KeyboardInterrupt: ignored

In [0]:
model2 = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features2 = model2.roi_heads.box_predictor.cls_score.in_features
model2.roi_heads.box_predictor = FastRCNNPredictor(in_features2, num_classes=2)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model2.to(device)
model2.load_state_dict(torch.load('drive/My Drive/cs-175-project/model_checkpoints/second_try_model.pt'))
dataset2 = PneumoniaDataset(root, True)
test_dataset2 = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset2)).tolist()
dataset2 = torch.utils.data.Subset(dataset2, indices[:-50])
data_loader2 = torch.utils.data.DataLoader(
        dataset2, batch_size=10, shuffle=True, num_workers=6,
        collate_fn=utils.collate_fn)

test_data_loader2 = torch.utils.data.DataLoader(
    test_dataset2, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)
params = [p for p in model2.parameters() if p.requires_grad]
optimizer2 = torch.optim.Adam(params, lr=0.00005, weight_decay=0.0005)
lr_step2 = torch.optim.lr_scheduler.StepLR(optimizer2,
                                               step_size=5,
                                               gamma=0.1)
for epoch in range(2):
    train_one_epoch(model2, optimizer2, data_loader2, device, epoch, print_freq=40)
    lr_step2.step()
    torch.save(model2.state_dict(), 'drive/My Drive/cs-175-project/model_checkpoints/second_try_model.pt')
    evaluate(model2, test_data_loader2, device=device)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch: [0]  [   0/2664]  eta: 2:50:31  lr: 0.000000  loss: 0.0765 (0.0765)  loss_classifier: 0.0391 (0.0391)  loss_box_reg: 0.0165 (0.0165)  loss_objectness: 0.0080 (0.0080)  loss_rpn_box_reg: 0.0128 (0.0128)  time: 3.8406  data: 1.4986  max mem: 7167
Epoch: [0]  [  40/2664]  eta: 1:26:05  lr: 0.000002  loss: 0.0421 (0.0520)  loss_classifier: 0.0219 (0.0255)  loss_box_reg: 0.0081 (0.0123)  loss_objectness: 0.0025 (0.0037)  loss_rpn_box_reg: 0.0078 (0.0106)  time: 1.9469  data: 0.0310  max mem: 7590
Epoch: [0]  [  80/2664]  eta: 1:24:49  lr: 0.000004  loss: 0.0487 (0.0495)  loss_classifier: 0.0245 (0.0258)  loss_box_reg: 0.0125 (0.0124)  loss_objectness: 0.0030 (0.0034)  loss_rpn_box_reg: 0.0046 (0.0080)  time: 1.9631  data: 0.0316  max mem: 7591
Epoch: [0]  [ 120/2664]  eta: 1:23:29  lr: 0.000006  loss: 0.0528 (0.0487)  loss_classifier: 0.0257 (0.0258)  loss_box_reg: 0.0121 (0.0127)  loss_objectness: 0.0033 (0.0033)  loss_rpn_box_reg: 0.0043 (0.0069)  time: 1.9673  data: 0.0314  max me

KeyboardInterrupt: ignored